In [82]:
import os
import pandas as pd
from openai import OpenAI


# Read data

In [83]:
#train_folder = "dataset/AdMIRe Subtask A Train/train/"
#train_df_file_path = train_folder + "subtask_a_train.tsv"

# train_folder = "dataset/AdMIRe Subtask A PT Train/"
# train_df_file_path = train_folder + "subtask_a_train.tsv"

#train_folder = "dataset/AdMIRe Subtask A Dev/dev/"
#train_df_file_path = train_folder + "subtask_a_dev.tsv"

#train_folder = "dataset/AdMIRe Subtask A PT Dev/dev/"
#train_df_file_path = train_folder + "subtask_a_dev.tsv"

# train_folder = "dataset/AdMIRe Subtask A Test/"
# train_df_file_path = train_folder + "subtask_a_test.tsv"

# train_folder = "dataset/AdMIRe Subtask A PT Test/"
# train_df_file_path = train_folder + "subtask_a_test.tsv"

# train_folder = "dataset/AdMIRe Subtask A PT Extended Evaluation/"
# train_df_file_path = train_folder + "subtask_a_xp.tsv"

train_folder = "dataset/AdMIRe Subtask A Extended Evaluation/"
train_df_file_path = train_folder + "subtask_a_xe.tsv"

df = pd.read_csv(train_df_file_path, sep='\t')


In [84]:
df

,compound,subset,sentence_type,sentence,expected_order,image1_name,image1_caption,image2_name,image2_caption,image3_name,image3_caption,image4_name,image4_caption,image5_name,image5_caption
0,elbow grease,Extended Evaluation,NaN,After a long day supporting herself with her a...,NaN,53956536828.png,The image depicts a hand holding a duster with...,54318499613.png,The image depicts a hand wearing a yellow work...,75007297939.png,The image depicts a hand holding a sponge and ...,83770618351.png,The image depicts a person wearing a black out...,89421546505.png,The image depicts a person wearing knee pads a...
1,night owl,Extended Evaluation,NaN,The photographer stayed up until midnight to c...,NaN,16314377152.png,The image depicts a cartoon-style illustration...,27691426039.png,The image depicts a nighttime scene with a lar...,55929836282.png,"The image depicts a dumbbell, which is a commo...",68674522585.png,The image depicts a cartoon-style illustration...,80731118574.png,The image depicts a cartoon-style owl perched ...
2,heart of gold,Extended Evaluation,NaN,Among the museum's rarest items they found a p...,NaN,52842957936.png,"The image depicts a large, metallic safe with ...",55746234435.png,"The image depicts a stylized, artistic represe...",60714765388.png,The image depicts a joyful scene featuring a y...,78172995823.png,"The image depicts a futuristic, stylized space...",83895016827.png,The image depicts a cartoon scene where a youn...
3,agony aunt,Extended Evaluation,NaN,The agony aunt crawls the ruins of the city wi...,NaN,01408794276.png,The image depicts a cartoon-style illustration...,21394025159.png,The image depicts a scene of a person sitting ...,40775868986.png,The image depicts a serene outdoor scene featu...,50587395734.png,The image depicts a cartoon character of a wom...,88126380876.png,"The image depicts a person sitting at a desk, ..."
4,shrinking violet,Extended Evaluation,NaN,"In her garden, she noticed a shrinking violet ...",NaN,23979985966.png,The image depicts an animated character with c...,25933704487.png,"The image depicts a small, cute puppy standing...",25967767410.png,The image is a colorful and vibrant illustrati...,43951540029.png,The image depicts a bouquet of purple tulips a...,55540866075.png,"The image depicts a magnifying glass, commonly..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,watering hole,Extended Evaluation,NaN,It fits in perfectly with the surroundings and...,NaN,12210905416.png,"The image depicts a small, quaint church with ...",21089764392.png,"The image depicts a small, fluffy guinea pig w...",45198868436.png,The image depicts a well-organized bar counter...,57736752320.png,"The image depicts a six-pack of beer bottles, ...",63209822903.png,The image depicts a vibrant and lively scene o...
96,flying saucer,Extended Evaluation,NaN,There is a display here featuring some of the ...,NaN,04578120988.png,"The image depicts a turntable, which is a devi...",17261577480.png,"The image depicts a cartoonish, anthropomorphi...",21183142808.png,"The image depicts a flying saucer, commonly as...",70585017093.png,The image depicts a white ceramic cup with an ...,97131856113.png,The image depicts a ceramic cup and saucer set...
97,act of god,Extended Evaluation,NaN,"Whether an event is intentional, accidental or...",NaN,12299908003.png,The image depicts a cartoon-style scene where ...,25535728219.png,The image depicts a vintage-style portable rad...,51312988406.png,The image depicts a whimsical scene featuring ...,65241916168.png,The image depicts a statue of a deity with mul...,80127117511.png,The image depicts a scene where a large tree h...
98,field work,Extended Evaluation,NaN,One of the highlights of the Mississippi Freed...,NaN,13420702948.png,The image depicts a cartoon character dressed ...,32932863559.png,The image depicts an elderly man kneeling down...,68928909822.png,The image depicts a cartoon scene involving th...,78594158224.png,The image depicts a ser

# Define a function to get an LLM response

In [85]:
api_key = "" # <=========================== Change the API key here

client = OpenAI(
    api_key=api_key,
)

# Define the model name dynamically
model_name = "gpt-4o"

def determine_sentence_type(message):
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "system",
              "content": "You are a professional linguist specializing in figurative language. You will be given a compound and a sentence containing this compound. Your task is determining whether this compound has an idiomatic or literal meaning in the sentence."
          },
          {
              "role": "user",
              "content": message,
          }
      ],
      model=model_name,
  )
  response_content = chat_completion.choices[0].message.content
  return response_content

def generate_idiom_meaning(message):
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "system",
              "content": "You are a professional linguist specializing in figurative language. You will be given an idiom. Your task is generating its meaning without any accompanying texts."
          },
          {
              "role": "user",
              "content": message,
          }
      ],
      model=model_name,
  )
  response_content = chat_completion.choices[0].message.content
  return response_content

# Generate idomatic meanings

In [86]:
meanings = []
sent_types = []
count = 0

from collections import Counter

def get_majority_response(query, n=5):
    responses = []
    for _ in range(n):
        response = determine_sentence_type(query).strip()
        if response in ["Literal", "Idiomatic"]:
            responses.append(response)
    print(responses)
    most_common_response, _ = Counter(responses).most_common(1)[0]
    return most_common_response

# Check if 'sentence_type' exists in the dataframe
has_sentence_type = 'sentence_type' in df.columns

for _, row in df.iterrows():
    compound = row['compound']
    sent_type = row['sentence_type']
    sent = row['sentence']

    # If 'sentence_type' exists, use it
    sent_type = row['sentence_type'] if has_sentence_type else None

    print(compound)
    print(sent)

    # First determine if the compound has an idiomatic or literal meaning based on a given sentence
    query = f"Given an example sentence: {sent}\nIs the phrase \"{compound}\" has literal or idiomatic meaning?\n" + "Only answer \"Literal\" or \"Idiomatic\"."
    response = get_majority_response(query, n=5)
    if sent_type == "idiomatic" and response.lower() == "idiomatic":
        count+=1
    elif sent_type == "literal" and response.lower() == "literal":
        count+=1
    print(sent_type, response.lower())

    # If it has an idiomatic meaning, then generate its meaning
    if response.lower() == 'idiomatic':
        query = f"Given an example sentence: {sent}\nWhat are the idiomatic meanings of {compound} in this example sentence?\n" + "Provide one answer wihtout accompanying texts."
        response = generate_idiom_meaning(query)
        print("idiomatic:", response)
        response.replace("\\", '').replace("'", "\\'").replace("\\\\'", "\\'")
        meaning = response
        meanings.append(meaning)
        sent_types.append("idiomatic")
    elif response.lower() == 'literal':
        meanings.append(None)
        sent_types.append("literal")
    else:
        raise NotImplemetedError

    print('')

# Save the generated meanings
df["meaning"] = meanings

# Save the determined compound type (idiomatic or literal)
if all(df["sentence_type"].isna()):
    df["sentence_type"] = sent_types

df["sent_type_predicted"] = sent_types

# Save data with a dynamically generated file name
output_file_name = train_df_file_path.replace(".tsv", f"_{model_name}_meanings.tsv")
df.to_csv(output_file_name, sep='\t', index=False)

elbow grease
After a long day supporting herself with her arms, she applied some elbow grease to soothe her dry, cracked skin.
['Literal', 'Literal', 'Literal', 'Literal', 'Literal']
nan literal

night owl
The photographer stayed up until midnight to capture a perfect shot of the elusive night owl.
['Literal', 'Literal', 'Literal', 'Literal', 'Literal']
nan literal

heart of gold
Among the museum's rarest items they found a priceless heart of gold, gleaming under the soft lights.
['Literal', 'Literal', 'Literal', 'Literal', 'Literal']
nan literal

agony aunt
The agony aunt crawls the ruins of the city with her nieces, harvesting the pain and suffering of the souls trapped there.
['Idiomatic', 'Idiomatic', 'Idiomatic', 'Idiomatic', 'Idiomatic']
nan idiomatic
idiomatic: In this context, "agony aunt" refers to a person who listens to, empathizes with, or helps others by addressing their emotional or personal problems, though the phrase is typically associated with someone who provides adv

In [87]:
# print(count/len(df))